# Controlling a Multi-Agent Team

This notebook covers how to control the lifecycle of a multi-agent team, including resetting, stopping, resuming, and aborting the team.


In [1]:
import asyncio
from rich import print as rprint
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Initialize the model client
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-2024-08-06",
)

# Define the agents with distinct roles
researcher = AssistantAgent(
    "researcher",
    model_client=model_client,
    system_message="You are a diligent researcher who gathers relevant information.",
)

analyst = AssistantAgent(
    "analyst",
    model_client=model_client,
    system_message="You analyze information and provide insightful summaries.",
)

writer = AssistantAgent(
    "writer",
    model_client=model_client,
    system_message="You write clear and concise reports based on analysis. Signal completion by including the word 'DONE'.",
)

# Define termination condition
termination_condition = TextMentionTermination("DONE")

# Create the team
team = RoundRobinGroupChat(
    [researcher, analyst, writer],
    termination_condition=termination_condition,
)

## Resetting the Team

Resetting clears the team's state and prepares it for a new task.


In [4]:
import nest_asyncio
nest_asyncio.apply()
from autogen_agentchat.conditions import ExternalTermination

async def reset_team():
    await team.reset()
    print("Team has been reset.")

## NOTE: we are running asynchronously
asyncio.run(reset_team())

Team has been reset.


## Stopping the Team Externally

You can stop the team from outside by using an external termination condition.


In [5]:
async def stop_team_externally():
    external_termination = ExternalTermination()
    team.termination_condition = external_termination

    run_task = asyncio.create_task(team.run(task="Write a short story about teamwork."))
    await asyncio.sleep(0.2)  # Let the team run briefly
    external_termination.set()  # Signal the team to stop
    await run_task
    print("Team stopped externally.")

# To execute, run in an async context:
asyncio.run(stop_team_externally())

Team stopped externally.


## Resuming the Team

Teams maintain state and can resume from where they left off.


In [6]:
async def resume_team():
    await team.reset()  # Reset first for demonstration
    await team.run(task="Start a brainstorming session on renewable energy.")
    print("Initial run complete.")

    # Resume without a new task
    await team.run()
    print("Team resumed and continued.")

# To execute, run in an async context:
asyncio.run(resume_team())

Initial run complete.
Team resumed and continued.


## Aborting the Team

Aborting immediately stops the team and raises a cancellation exception.

Even though we catch the exception, Jupyter will still print it.

In [17]:
from autogen_core import CancellationToken
import asyncio

async def abort_team():
    cancellation_token = CancellationToken()
    run_task = asyncio.create_task(
        team.run(task="Generate a detailed report on climate change.", cancellation_token=cancellation_token)
    )
    await asyncio.sleep(0.1)  # Let the team start
    cancellation_token.cancel()  # Abort the team
    try:
        await run_task
    except asyncio.exceptions.CancelledError:
        print("Team run was aborted.")

# To execute, run in an async context:
asyncio.run(abort_team())

Error processing publish message for analyst_a14046b4-f488-4270-a73a-0e67a09086f1/a14046b4-f488-4270-a73a-0e67a09086f1
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 606, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/autogen_core/_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/autogen_core/_routed_agent.py", line 485, in on_message_impl
    return await h(self, message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/

Team run was aborted.
